# Keras tranfer learning dog breeds classifier

In [16]:
import numpy as np
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
base_model = MobileNet(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(133, activation='softmax')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=preds)

In [8]:
for i, layer in enumerate(model.layers):
    print(i, layer, layer.trainable)

0 <keras.engine.input_layer.InputLayer object at 0x11082e4e0> False
1 <keras.layers.convolutional.ZeroPadding2D object at 0x632314048> False
2 <keras.layers.convolutional.Conv2D object at 0x632314390> False
3 <keras.layers.normalization.BatchNormalization object at 0x632314588> False
4 <keras.layers.advanced_activations.ReLU object at 0x632314c88> False
5 <keras.layers.convolutional.DepthwiseConv2D object at 0x6323145c0> False
6 <keras.layers.normalization.BatchNormalization object at 0x632368080> False
7 <keras.layers.advanced_activations.ReLU object at 0x6323ff908> False
8 <keras.layers.convolutional.Conv2D object at 0x6324ae908> False
9 <keras.layers.normalization.BatchNormalization object at 0x63246aeb8> False
10 <keras.layers.advanced_activations.ReLU object at 0x63242b588> False
11 <keras.layers.convolutional.ZeroPadding2D object at 0x63250b160> False
12 <keras.layers.convolutional.DepthwiseConv2D object at 0x63250b9b0> False
13 <keras.layers.normalization.BatchNormalization obje

In [9]:
for layer in model.layers[:87]:
    layer.trainable=False

In [10]:
image_size = (224, 224)
batch_size = 32
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dogImages/train',
    target_size=image_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_generator = valid_datagen.flow_from_directory(
    'dogImages/valid',
    target_size=image_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    'dogImages/test',
    target_size=image_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical'
)

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


In [26]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='keras_tl_dbc.hdf5', 
    save_best_only=True,
    verbose=1
)

In [31]:
step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    steps_per_epoch=step_size_train,
    callbacks=[checkpointer],
    epochs=5
)

Epoch 1/5
208/208 [==============================] - 508s 2s/step - loss: 1.0086 - accuracy: 0.6779 - val_loss: 1.0664 - val_accuracy: 0.6898

Epoch 00001: val_loss improved from inf to 1.06643, saving model to keras_tl_dbc.hdf5
Epoch 2/5
208/208 [==============================] - 487s 2s/step - loss: 0.7988 - accuracy: 0.7435 - val_loss: 0.7524 - val_accuracy: 0.7102

Epoch 00002: val_loss improved from 1.06643 to 0.75235, saving model to keras_tl_dbc.hdf5
Epoch 3/5
208/208 [==============================] - 468s 2s/step - loss: 0.6285 - accuracy: 0.7911 - val_loss: 1.6260 - val_accuracy: 0.7293

Epoch 00003: val_loss did not improve from 0.75235
Epoch 4/5
208/208 [==============================] - 473s 2s/step - loss: 0.5179 - accuracy: 0.8281 - val_loss: 0.8941 - val_accuracy: 0.7449

Epoch 00004: val_loss did not improve from 0.75235
Epoch 5/5
208/208 [==============================] - 480s 2s/step - loss: 0.3951 - accuracy: 0.8673 - val_loss: 0.0106 - val_accuracy: 0.7054

Epoch 0

In [30]:
model.load_weights('keras_tl_dbc.hdf5')
# get index of predicted dog breed for each image in test set
loss, acc = model.evaluate_generator(test_generator, verbose=1)
print('Test accuracy: %.4f%%' % (acc * 100))

27/27 [==============================] - 59s 2s/step
Test accuracy: 72.0096%


In [24]:
from tqdm import tqdm_notebook as tqdm
[np.argmax(test_generator.next()[1]) for iter in tqdm(range(len(test_generator)))]

/Users/alexchernov/anaconda3/envs/deep-learning/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[55,
 111,
 72,
 85,
 99,
 49,
 88,
 78,
 11,
 87,
 1,
 44,
 77,
 85,
 108,
 69,
 66,
 45,
 84,
 31,
 33,
 55,
 89,
 121,
 131,
 77,
 84]